Advanced Analytics
Assignment 1
Group 11
Peter Day

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
# https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

/home/muddy/.local/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Column names
['client_id', 'homebanking_active', 'has_homebanking', 'has_insurance_21', 'has_insurance_23', 'has_life_insurance_fixed_cap', 'has_life_insurance_decreasing_cap', 'has_fire_car_other_insurance', 'has_personal_loan', 'has_mortgage_loan', 'has_current_account', 'has_pension_saving', 'has_savings_account', 'has_savings_account_starter', 'has_current_account_starter', 'bal_insurance_21', 'bal_insurance_23', 'cap_life_insurance_fixed_cap', 'cap_life_insurance_decreasing_cap', 'prem_fire_car_other_insurance', 'bal_personal_loan', 'bal_mortgage_loan', 'bal_current_account', 'bal_pension_saving', 'bal_savings_account', 'bal_savings_account_starter', 'bal_current_account_starter', 'visits_distinct_so', 'visits_distinct_so_areas', 'customer_since_all', 'customer_since_bank', 'customer_gender', 'customer_birth_date', 'customer_postal_code', 'customer_occupation_code', 'customer_self_employed', 'customer_education', 'customer_children', 'customer_relationship', 'target']


https://www.statology.org/pandas-rename-columns/


In [2]:
# csv data files are in a folder called data
file1 = "../data/train_month_1.csv"
file2 = "../data/train_month_2.csv"
file3 = "../data/train_month_3_with_target.csv"
files = [file3, file2, file1]
# https://stackoverflow.com/questions/68607106/creating-multiple-dataframes-using-for-loop-with-pandas
# my_dfs = {}
# for x in ['A', 'B', 'C']:
#     my_dfs[x] = pd.read_csv(r"C:\HSTS\OB\ODO\%s\test.csv" % x, delimiter=';')
# Then access the dataframes per key:
# my_dfs['A']

listnames = ['data3', 'data2', 'data1']
my_dfs = {}
for i in range(3):
    my_dfs[listnames[i]] = pd.read_csv(files[i])


In [3]:
# https://www.tutorialspoint.com/How-to-iterate-through-a-dictionary-in-Python
cols = ["customer_since_all", "customer_since_bank", "customer_birth_date"]
# for column in stu_df[['Name', 'Section']]:
# df[col] = pd.to_datetime(df[col])

# This converts the 3 time cols in the 3 dfs to datetime format then subtracts 'current date' to get time since
for k, v in my_dfs.items():
    print(k)
    for col in cols:
        v[col] = (pd.to_datetime('2018-03-01') - pd.to_datetime(v[col])).dt.days
        #v[col] = v[col].astype(int)
        
# if you run this more than once it gets confused, will have to start over by running the cell above too!

data3
data2
data1


In [4]:
data3 = my_dfs['data3']
data2 = my_dfs['data2']
data1 = my_dfs['data1']

In [5]:
# Rename columns with lag suffix
data3.columns = ['client_id', 'homebanking_active_l0', 'has_homebanking_l0', 'has_insurance_21_l0', 'has_insurance_23_l0', 'has_life_insurance_fixed_cap_l0', 'has_life_insurance_decreasing_cap_l0', 'has_fire_car_other_insurance_l0', 'has_personal_loan_l0', 'has_mortgage_loan_l0', 'has_current_account_l0', 'has_pension_saving_l0', 'has_savings_account_l0', 'has_savings_account_starter_l0', 'has_current_account_starter_l0', 'bal_insurance_21_l0', 'bal_insurance_23_l0', 'cap_life_insurance_fixed_cap_l0', 'cap_life_insurance_decreasing_cap_l0', 'prem_fire_car_other_insurance_l0', 'bal_personal_loan_l0', 'bal_mortgage_loan_l0', 'bal_current_account_l0', 'bal_pension_saving_l0', 'bal_savings_account_l0', 'bal_savings_account_starter_l0', 'bal_current_account_starter_l0', 'visits_distinct_so_l0', 'visits_distinct_so_areas_l0', 'customer_since_all_l0', 'customer_since_bank_l0', 'customer_gender_l0', 'customer_birth_date_l0', 'customer_postal_code_l0', 'customer_occupation_code_l0', 'customer_self_employed_l0', 'customer_education_l0', 'customer_children_l0', 'customer_relationship_l0', 'target']
data2.columns = ['client_id', 'homebanking_active_l1', 'has_homebanking_l1', 'has_insurance_21_l1', 'has_insurance_23_l1', 'has_life_insurance_fixed_cap_l1', 'has_life_insurance_decreasing_cap_l1', 'has_fire_car_other_insurance_l1', 'has_personal_loan_l1', 'has_mortgage_loan_l1', 'has_current_account_l1', 'has_pension_saving_l1', 'has_savings_account_l1', 'has_savings_account_starter_l1', 'has_current_account_starter_l1', 'bal_insurance_21_l1', 'bal_insurance_23_l1', 'cap_life_insurance_fixed_cap_l1', 'cap_life_insurance_decreasing_cap_l1', 'prem_fire_car_other_insurance_l1', 'bal_personal_loan_l1', 'bal_mortgage_loan_l1', 'bal_current_account_l1', 'bal_pension_saving_l1', 'bal_savings_account_l1', 'bal_savings_account_starter_l1', 'bal_current_account_starter_l1', 'visits_distinct_so_l1', 'visits_distinct_so_areas_l1', 'customer_since_all_l1', 'customer_since_bank_l1', 'customer_gender_l1', 'customer_birth_date_l1', 'customer_postal_code_l1', 'customer_occupation_code_l1', 'customer_self_employed_l1', 'customer_education_l1', 'customer_children_l1', 'customer_relationship_l1']
data1.columns = ['client_id', 'homebanking_active_l2', 'has_homebanking_l2', 'has_insurance_21_l2', 'has_insurance_23_l2', 'has_life_insurance_fixed_cap_l2', 'has_life_insurance_decreasing_cap_l2', 'has_fire_car_other_insurance_l2', 'has_personal_loan_l2', 'has_mortgage_loan_l2', 'has_current_account_l2', 'has_pension_saving_l2', 'has_savings_account_l2', 'has_savings_account_starter_l2', 'has_current_account_starter_l2', 'bal_insurance_21_l2', 'bal_insurance_23_l2', 'cap_life_insurance_fixed_cap_l2', 'cap_life_insurance_decreasing_cap_l2', 'prem_fire_car_other_insurance_l2', 'bal_personal_loan_l2', 'bal_mortgage_loan_l2', 'bal_current_account_l2', 'bal_pension_saving_l2', 'bal_savings_account_l2', 'bal_savings_account_starter_l2', 'bal_current_account_starter_l2', 'visits_distinct_so_l2', 'visits_distinct_so_areas_l2', 'customer_since_all_l2', 'customer_since_bank_l2', 'customer_gender_l2', 'customer_birth_date_l2', 'customer_postal_code_l2', 'customer_occupation_code_l2', 'customer_self_employed_l2', 'customer_education_l2', 'customer_children_l2', 'customer_relationship_l2']

In [6]:
# Join the 3 data frames into one.
data4 = data3.merge(data2, on='client_id', how='inner')
data_Nans = data4.merge(data1, on='client_id', how='inner')


In [7]:
# Impute missing values
#imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#imputer.fit(data)
#data = pd.DataFrame(data=imputer.transform(data_Nans), columns=data_Nans.columns)

In [8]:
# Impute without target for 115 columns to use with test data
data_Nans = data_Nans.drop(columns='target', axis=1)
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data_Nans)
data = pd.DataFrame(data=imputer.transform(data_Nans), columns=data_Nans.columns)

In [9]:
data.customer_children_l0.unique()

array(['no', 'mature', 'young', 'preschool', 'adolescent', 'grownup',
       'onebaby', 'yes'], dtype=object)

In [10]:
data.customer_relationship_l0.unique()

array(['couple', 'single'], dtype=object)

In [11]:
#df.edjefe.replace(to_replace=['no', 'yes'], value=[0, 1])
data.replace(to_replace=['no', 'mature', 'young', 'preschool', 'adolescent', 'grownup', 'onebaby', 'yes'],
             value=[0, 1, 1, 1, 1, 1, 1, 1], inplace=True)
data.replace(to_replace=['couple', 'single'], value=[1, 0], inplace=True)

In [12]:
data.shape

(63697, 115)

In [13]:
y = data3['target'].copy()


In [14]:
# https://www.statology.org/drop-column-pandas/
X = data.drop(columns='client_id', axis=1)
# scaler = StandardScaler()
# fit = scaler.fit(X)
# Z = fit.transform(X)

In [15]:
def sensitivity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sens = cm[1][1]/sum(cm[1])
    return(sens)
def pospredvalue(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if (cm[0][1]+cm[1][1]) == 0:
        return(0.0)
    else:
        ppv = cm[1][1]/(cm[0][1]+cm[1][1])
        return(ppv)
def totalfalserate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tfr = (cm[0][1]+cm[1][0])/sum(sum(cm))
    return(tfr)

In [16]:
# Split data for training/validation, assuming balanced classes
# by default 75/25 split.  33 is a rndm seed - for reproducibility
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33)

In [17]:
#X_train.head()

In [18]:
#X_train.describe()
# this ignores columns that aren't purely numeric, including dates, but does include postal code

In [19]:
# try a RF fit
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train, y_train)
#rf.score(X_valid, y_valid)

RandomForestClassifier(max_depth=2, random_state=0)

In [20]:
rf.score(X_valid, y_valid)

0.9697331240188383

In [21]:
y_predrf = rf.predict(X_valid)
print(confusion_matrix(y_valid, y_predrf))

[[15443     0]
 [  482     0]]


In [22]:
sensitivity(y_valid, y_predrf)

0.0

In [23]:
pospredvalue(y_valid, y_predrf)

0.0

In [24]:
# sklearn.metrics.roc_auc_score(y_true, y_score, *, average='macro', sample_weight=None, max_fpr=None, multi_class='raise', labels=None)
#y_scorerf = rf.predic_proba(X_valid)
#roc_auc_score(y_valid, y_scorerf)
# AttributeError: 'RandomForestClassifier' object has no attribute 'predic_proba'

In [47]:
# https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn/notebook
# Try xgboost classifier
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric = ['auc', 'map'],
    random_state=33
)
xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-auc:0.67329	validation_0-map:0.06649
[1]	validation_0-auc:0.69718	validation_0-map:0.07453
[2]	validation_0-auc:0.70382	validation_0-map:0.07550
[3]	validation_0-auc:0.71688	validation_0-map:0.07638
[4]	validation_0-auc:0.71843	validation_0-map:0.07555
[5]	validation_0-auc:0.72110	validation_0-map:0.07674
[6]	validation_0-auc:0.72077	validation_0-map:0.07820
[7]	validation_0-auc:0.72576	validation_0-map:0.08190
[8]	validation_0-auc:0.72599	validation_0-map:0.08082
[9]	validation_0-auc:0.72352	validation_0-map:0.08199
[10]	validation_0-auc:0.72131	validation_0-map:0.08155
[11]	validation_0-auc:0.72590	validation_0-map:0.08320
[12]	validation_0-auc:0.72380	validation_0-map:0.08175
[13]	validation_0-auc:0.72457	validation_0-map:0.08225
[14]	validation_0-auc:0.72492	validation_0-map:0.08384
[15]	validation_0-auc:0.72451	validation_0-map:0.08273
[16]	validation_0-auc:0.72640	validation_0-map:0.08295
[17]	validation_0-auc:0.72653	validation_0-map:0.08308
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric=['auc', 'map'], gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=33, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [48]:
xgb.score(X_valid, y_valid)

0.9696075353218211

In [49]:
y_predxgb = xgb.predict(X_valid)
print(confusion_matrix(y_valid, y_predxgb))

[[15439     4]
 [  480     2]]


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [50]:
tn, fp, fn, tp = confusion_matrix(y_valid, y_predxgb).ravel()
(tn, fp, fn, tp)

(15439, 4, 480, 2)

In [39]:
sensitivity(y_valid, y_predxgb)

0.004149377593360996

In [40]:
totalfalserate(y_valid, y_predxgb)

0.030392464678178965

In [33]:
# really slow: SVC(kernel="rbf", C=0.025, probability=True),
classifiers = [
    RandomForestClassifier(max_depth=5, random_state=0)
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    MLPClassifier((20,20), random_state=33, max_iter=500)
    ]

In [34]:
for classifier in classifiers:
    classifier.fit(X_train, y_train)   
    y_pred = classifier.predict(X_valid)
    print(classifier)
    print("sensitivity: %.4f" % sensitivity(y_valid, y_pred))
    print("total false rate: %.4f" % totalfalserate(y_valid, y_pred))
    print("-------------------------")

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier(n_neighbors=3)
sensitivity: 0.0062
total false rate: 0.0341
-------------------------
DecisionTreeClassifier()
sensitivity: 0.0954
total false rate: 0.0598
-------------------------
RandomForestClassifier()
sensitivity: 0.0000
total false rate: 0.0303
-------------------------


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


AdaBoostClassifier()
sensitivity: 0.0000
total false rate: 0.0303
-------------------------
GradientBoostingClassifier()
sensitivity: 0.0021
total false rate: 0.0307
-------------------------
MLPClassifier(hidden_layer_sizes=(20, 20), max_iter=500, random_state=33)
sensitivity: 0.0000
total false rate: 0.0304
-------------------------


In [45]:
for i in range(1, 10, 2):
    kn = KNeighborsClassifier(i)
    kn.fit(X_train, y_train)
    y_pred = kn.predict(X_valid)
    print(kn)
    print("sensitivity: %.4f" % sensitivity(y_valid, y_pred))
    print("total false rate: %.4f" % totalfalserate(y_valid, y_pred))
    print("-------------------------")

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier(n_neighbors=1)
sensitivity: 0.0519
total false rate: 0.0550
-------------------------


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier(n_neighbors=3)
sensitivity: 0.0062
total false rate: 0.0341
-------------------------


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier()
sensitivity: 0.0021
total false rate: 0.0309
-------------------------


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier(n_neighbors=7)
sensitivity: 0.0021
total false rate: 0.0306
-------------------------


c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNeighborsClassifier(n_neighbors=9)
sensitivity: 0.0000
total false rate: 0.0303
-------------------------


## Now predict on test data

In [33]:
file1_test = "../data/test_month_1.csv"
file2_test = "../data/test_month_2.csv"
file3_test = "../data/test_month_3.csv"
files_test = [file3_test, file2_test, file1_test]

listnames_test = ['data3_test', 'data2_test', 'data1_test']
my_dfs_test = {}
for i in range(3):
    my_dfs_test[listnames_test[i]] = pd.read_csv(files_test[i])


In [34]:
# This converts the 3 time cols in the 3 dfs to datetime format then subtracts 'current date' to get time since
cols_test = ["customer_since_all", "customer_since_bank", "customer_birth_date"]
for k, v in my_dfs_test.items():
    print(k)
    print(len(v))
    for col in cols_test:
        v[col] = (pd.to_datetime('2018-03-01') - pd.to_datetime(v[col])).dt.days

data3_test
27300
data2_test
27300
data1_test
27300


In [35]:
data3_test = my_dfs_test['data3_test']
data2_test = my_dfs_test['data2_test']
data1_test = my_dfs_test['data1_test']


In [36]:
# Rename columns with lag suffix
data3_test.columns = ['client_id', 'homebanking_active_l0', 'has_homebanking_l0', 'has_insurance_21_l0', 'has_insurance_23_l0', 'has_life_insurance_fixed_cap_l0', 'has_life_insurance_decreasing_cap_l0', 'has_fire_car_other_insurance_l0', 'has_personal_loan_l0', 'has_mortgage_loan_l0', 'has_current_account_l0', 'has_pension_saving_l0', 'has_savings_account_l0', 'has_savings_account_starter_l0', 'has_current_account_starter_l0', 'bal_insurance_21_l0', 'bal_insurance_23_l0', 'cap_life_insurance_fixed_cap_l0', 'cap_life_insurance_decreasing_cap_l0', 'prem_fire_car_other_insurance_l0', 'bal_personal_loan_l0', 'bal_mortgage_loan_l0', 'bal_current_account_l0', 'bal_pension_saving_l0', 'bal_savings_account_l0', 'bal_savings_account_starter_l0', 'bal_current_account_starter_l0', 'visits_distinct_so_l0', 'visits_distinct_so_areas_l0', 'customer_since_all_l0', 'customer_since_bank_l0', 'customer_gender_l0', 'customer_birth_date_l0', 'customer_postal_code_l0', 'customer_occupation_code_l0', 'customer_self_employed_l0', 'customer_education_l0', 'customer_children_l0', 'customer_relationship_l0']
data2_test.columns = ['client_id', 'homebanking_active_l1', 'has_homebanking_l1', 'has_insurance_21_l1', 'has_insurance_23_l1', 'has_life_insurance_fixed_cap_l1', 'has_life_insurance_decreasing_cap_l1', 'has_fire_car_other_insurance_l1', 'has_personal_loan_l1', 'has_mortgage_loan_l1', 'has_current_account_l1', 'has_pension_saving_l1', 'has_savings_account_l1', 'has_savings_account_starter_l1', 'has_current_account_starter_l1', 'bal_insurance_21_l1', 'bal_insurance_23_l1', 'cap_life_insurance_fixed_cap_l1', 'cap_life_insurance_decreasing_cap_l1', 'prem_fire_car_other_insurance_l1', 'bal_personal_loan_l1', 'bal_mortgage_loan_l1', 'bal_current_account_l1', 'bal_pension_saving_l1', 'bal_savings_account_l1', 'bal_savings_account_starter_l1', 'bal_current_account_starter_l1', 'visits_distinct_so_l1', 'visits_distinct_so_areas_l1', 'customer_since_all_l1', 'customer_since_bank_l1', 'customer_gender_l1', 'customer_birth_date_l1', 'customer_postal_code_l1', 'customer_occupation_code_l1', 'customer_self_employed_l1', 'customer_education_l1', 'customer_children_l1', 'customer_relationship_l1']
data1_test.columns = ['client_id', 'homebanking_active_l2', 'has_homebanking_l2', 'has_insurance_21_l2', 'has_insurance_23_l2', 'has_life_insurance_fixed_cap_l2', 'has_life_insurance_decreasing_cap_l2', 'has_fire_car_other_insurance_l2', 'has_personal_loan_l2', 'has_mortgage_loan_l2', 'has_current_account_l2', 'has_pension_saving_l2', 'has_savings_account_l2', 'has_savings_account_starter_l2', 'has_current_account_starter_l2', 'bal_insurance_21_l2', 'bal_insurance_23_l2', 'cap_life_insurance_fixed_cap_l2', 'cap_life_insurance_decreasing_cap_l2', 'prem_fire_car_other_insurance_l2', 'bal_personal_loan_l2', 'bal_mortgage_loan_l2', 'bal_current_account_l2', 'bal_pension_saving_l2', 'bal_savings_account_l2', 'bal_savings_account_starter_l2', 'bal_current_account_starter_l2', 'visits_distinct_so_l2', 'visits_distinct_so_areas_l2', 'customer_since_all_l2', 'customer_since_bank_l2', 'customer_gender_l2', 'customer_birth_date_l2', 'customer_postal_code_l2', 'customer_occupation_code_l2', 'customer_self_employed_l2', 'customer_education_l2', 'customer_children_l2', 'customer_relationship_l2']

# Join the 3 data frames into one.
data4_test = data3_test.merge(data2_test, on='client_id', how='inner')
data_Nans_test = data4_test.merge(data1_test, on='client_id', how='inner')

In [40]:
# Impute missing values
data_test = pd.DataFrame(data=imputer.transform(data_Nans_test), columns=data_Nans_test.columns)
data_test.replace(to_replace=['no', 'mature', 'young', 'preschool', 'adolescent', 'grownup', 'onebaby', 'yes'],
             value=[0, 1, 1, 1, 1, 1, 1, 1], inplace=True)
data_test.replace(to_replace=['couple', 'single'], value=[1, 0], inplace=True)
clientID = pd.DataFrame(data=data_test['client_id'])
X_test = data_test.drop(columns='client_id', axis=1)

## Predict on test data and save as csv

In [50]:
y_pred_test = pd.DataFrame(xgb.predict_proba(X_test))
# Decision Tree did worse on the test data
#y_pred_test = pd.DataFrame(dt.predict_proba(X_test))

In [51]:
print(y_pred_test)

              0         1
0      0.984721  0.015279
1      0.970955  0.029045
2      0.918003  0.081997
3      0.994368  0.005632
4      0.978663  0.021337
...         ...       ...
27295  0.979317  0.020683
27296  0.927633  0.072367
27297  0.984245  0.015755
27298  0.974884  0.025116
27299  0.982389  0.017611

[27300 rows x 2 columns]


In [52]:
clientID['pred']=y_pred_test[1]

In [53]:
clientID.to_csv("y_pred_test.csv")